In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
"""
first, we need to import all the data in ov_datasets.csv
which has gathered all tumor volume data in McCart 2021 et al. with webplotdigitizer

data structure:
1. the first 20 columns are the tumor volume under vvDD condition
   specifying x (time) and y (tumor volume) for each two columns, totalling 10 samples
   the first row can be ignored
   the 1-5 rows are the 5 time points for all 10 samples
2. the next 20 columns are the tumor volume under normal/PBS condition with similar structure
3. the last four columns are the mean and stde data of the tumor volume under vvDD condition and normal/PBS condition
   each two columns specify one condition, respectively the x and y
   each three rows specify one time point: mean, stde+, and stde-
   again, the first row can be ignored

"""

df = pd.read_csv('/Users/yuhongliu/Documents/OV/data/ov_datasets_v7.csv')

# Extract tumor volume data for vvDD condition
tumor_vol_vvDD = df.iloc[1:6, 1:20:2].values  # Select every other column starting from the second column (Y values)
# Create a DataFrame with known time points and extracted tumor volumes
tumor_vol_vvDD_df = pd.DataFrame(tumor_vol_vvDD, index=[0, 1, 2, 3, 4], columns=[f'vvDD_sample_{i+1}' for i in range(10)])

# Extract tumor volume data for normal/PBS condition
tumor_vol_pbs = df.iloc[1:6, 21:40:2].values  # Select every other column starting from the 21 column (Y values)
# Create a DataFrame with known time points and extracted tumor volumes
tumor_vol_pbs_df = pd.DataFrame(tumor_vol_pbs, index=[0, 1, 2, 3, 4], columns=[f'pbs_sample_{i+1}' for i in range(10)])

# Extract mean and standard deviation data for vvDD and PBS conditions, for comparison with the two dataframes above
vvDD_mean_stde_data = df.iloc[1:, -3].values
pbs_mean_stde_data = df.iloc[1:, -1].values
# Create DataFrames for vvDD and PBS data
vvDD_mean_stde_df = pd.DataFrame(vvDD_mean_stde_data.reshape(-1, 3), 
                                 index=[0, 1, 2, 3, 4], 
                                 columns=['mean', 'stde_plus', 'stde_minus'])

pbs_mean_stde_df = pd.DataFrame(pbs_mean_stde_data.reshape(-1, 3), 
                                index=[0, 1, 2, 3, 4], 
                                columns=['mean', 'stde_plus', 'stde_minus'])

# convert data to numeric type
tumor_vol_vvDD_df = tumor_vol_vvDD_df.apply(pd.to_numeric, errors='coerce')
tumor_vol_pbs_df = tumor_vol_pbs_df.apply(pd.to_numeric, errors='coerce')
vvDD_mean_stde_df = vvDD_mean_stde_df.apply(pd.to_numeric, errors='coerce')
pbs_mean_stde_df = pbs_mean_stde_df.apply(pd.to_numeric, errors='coerce')


In [ ]:
tumor_vol_vvDD_df

In [ ]:
# plot the tumor volume data for vvDD and PBS condition
plt.figure(figsize=(10, 6))

# Plot vvDD data
for i in range(10):
    plt.plot(tumor_vol_vvDD_df.index, tumor_vol_vvDD_df.iloc[:, i], 
             marker='o', linestyle='-', color='blue', alpha=0.3, label='vvDD' if i == 0 else "")

# Plot PBS data
for i in range(10):
    plt.plot(tumor_vol_pbs_df.index, tumor_vol_pbs_df.iloc[:, i], 
             marker='o', linestyle='-', color='red', alpha=0.3, label='PBS' if i == 0 else "")

plt.xlabel('Time (days)')
plt.ylabel('Tumor Volume ($\mu m^3$)')
plt.title('Tumor Volume vs Time for vvDD and PBS Conditions')
plt.legend()
plt.grid(False)
plt.xticks([0, 1, 2, 3, 4], [3, 4, 5, 6, 7])
plt.show()

In [ ]:
# plot the mean and stde data for vvDD and PBS condition

# Calculate mean and standard error for vvDD
vvDD_mean = tumor_vol_vvDD_df.mean(axis=1)
vvDD_se = tumor_vol_vvDD_df.sem(axis=1)
# Calculate mean and standard error for PBS
pbs_mean = tumor_vol_pbs_df.mean(axis=1)
pbs_se = tumor_vol_pbs_df.sem(axis=1)

# Plot both provided and calculated data
plt.figure(figsize=(12, 6))
# Plot provided data
plt.errorbar(vvDD_mean_stde_df.index, vvDD_mean_stde_df['mean'], 
             yerr=[vvDD_mean_stde_df['mean'] - vvDD_mean_stde_df['stde_minus'], 
                   vvDD_mean_stde_df['stde_plus'] - vvDD_mean_stde_df['mean']],
             fmt='o-', color='blue', ecolor='lightblue', capsize=5, label='vvDD (provided)')
plt.errorbar(pbs_mean_stde_df.index, pbs_mean_stde_df['mean'], 
             yerr=[pbs_mean_stde_df['mean'] - pbs_mean_stde_df['stde_minus'], 
                   pbs_mean_stde_df['stde_plus'] - pbs_mean_stde_df['mean']],
             fmt='o-', color='red', ecolor='lightcoral', capsize=5, label='PBS (provided)')
# Plot calculated data
plt.errorbar(vvDD_mean.index, vvDD_mean, yerr=vvDD_se,
             fmt='s--', color='darkblue', ecolor='skyblue', capsize=5, label='vvDD (calculated)')
plt.errorbar(pbs_mean.index, pbs_mean, yerr=pbs_se,
             fmt='s--', color='darkred', ecolor='lightpink', capsize=5, label='PBS (calculated)')
plt.xlabel('Time (days)')
plt.ylabel('Tumor Volume (mm^3)')
plt.title('Mean Tumor Volume with Standard Error vs Time')
plt.legend()
plt.grid(False)
plt.xticks([3, 4, 5, 6, 7])
plt.show()



In [7]:
# make the measurement.tsv file
def create_rows(df, condition):
    rows = []
    for time in df.index:
        for col in df.columns:
            rows.append({
                'observableId': 'tumor_vol',
                'simulationConditionId': condition,
                'measurement': df.loc[time, col],
                'time': time,
                'observableParameters': 'scaling_1',
                'noiseParameters': 'sd_vol'
            })
    return rows

# Create rows for vvDD and PBS conditions
vvDD_rows = create_rows(tumor_vol_vvDD_df, 'vvDD')
pbs_rows = create_rows(tumor_vol_pbs_df, 'ctrl')

# Combine all rows
all_rows = vvDD_rows + pbs_rows

# Create the final dataframe
measurements_df = pd.DataFrame(all_rows)

# Reorder columns to match the specified order
column_order = ['observableId', 'simulationConditionId', 'measurement', 'time', 'observableParameters', 'noiseParameters']
measurements_df = measurements_df[column_order]

# Save to TSV file
measurements_df.to_csv('petab_files/measurements.tsv', sep='\t', index=False)

In [8]:
# 1. Create the model file using antimony
# this is being done in model_creation.py

# 2. Create the parameter table
parameter_df = pd.DataFrame({
    'parameterId': ['rho', 'kappa', 'psi', 'phi', 'beta', 'alpha', 'delta', 'scaling_1', 'sd_vol'],
    'parameterName': ['rho', 'kappa', 'psi', 'phi', 'beta', 'alpha', 'delta', 'scaling_1', 'sd_vol'],
    'parameterScale': ['log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'log10', 'lin', 'lin'],
    'lowerBound': [1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 1e-4, 0],
    'upperBound': [1e4, 1e4, 1e4, 1e4, 1e4, 1e4, 1e4, 1e4, 'inf'],
    'nominalValue': [1] * 9,
    'estimate': [1] * 9,
    'parameterType': ['', '', '', '', '', '', '', 'scaling', 'sigma']
})
# export the parameter table to tsv file
parameter_df.to_csv('petab_files/parameters_v2.tsv', sep='\t', index=False)

# 3. Create the observable table
hidden = True
n_hidden = 5  # Define the number of hidden variables

if hidden:
    observable_formula = 'observableParameter1_tumor_vol * (C_u + C_i + ' + ' + '.join([f'C_i{j}' for j in range(1, n_hidden + 1)]) + ' + C_l)'
    observable_df = pd.DataFrame({
        'observableId': ['tumor_vol'],
        'observableName': ['tumor_volume'],
        'observableFormula': [observable_formula],
        'noiseFormula': ['noiseParameter1_tumor_vol'],
        'noiseDistribution': ['normal'],
        'observableTransformation': ['lin']
    })
else:
    observable_df = pd.DataFrame({
        'observableId': ['tumor_vol'],
        'observableName': ['tumor_volume'],
        'observableFormula': ['observableParameter1_tumor_vol * (C_u + C_i)'],
        'noiseFormula': ['noiseParameter1_tumor_vol'],
        'noiseDistribution': ['normal'],
        'observableTransformation': ['lin']
    })

# export the observable table to tsv file
observable_df.to_csv('petab_files/observables_v2.tsv', sep='\t', index=False)

# 4. Create the measurement table
# this is being done above

# 5. Create the condition table
condition_df = pd.DataFrame({
    'conditionId': ['vvDD', 'ctrl'],
    'conditionName': ['vvDD', 'ctrl'],
    'virus_injection': [3e9, 0]
})
# export the condition table to tsv file
condition_df.to_csv('petab_files/conditions.tsv', sep='\t', index=False)
